In [1]:
%pip install langgraph langchain_openai langfuse "smolagents[types]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from smolagents import LiteLLMModel
from langchain.schema import BaseMessage, ChatMessage

In [3]:
class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    spam_reason: Optional[str]
    email_category: Optional[str]
    email_draft: Optional[str]
    messages: List[Dict[str, Any]]

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash-lite",
    temperature=0.0,
    max_tokens=1024
)

In [4]:
def read_email(state: EmailState):
    """Agent reads and logs the incoming email"""
    email = state['email']
    # Here we might do some initial preprocessing
    print(f"Alfred is processing an email from {email['sender']} with subject: {email['subject']}")

    return{}

def classify_email(state: EmailState):
    """Agent uses an LLM to determine whether an email is spam or legitamate"""
    e = state["email"]

    # Ask for structured output to make parsing robust and avoid substring traps.
    prompt = f"""
    You are an email triage agent. Read the email and return a JSON object with:
    - "is_spam": true|false
    - "spam_reason": string or null
    - "category": one of ["inquiry","complaint","thank-you","request","info"] or null

    Email:
    From: {e['sender']}
    Subject: {e['subject']}
    Body: {e['body']}
    Return JSON only.
    """.strip()

    messages = [
        ChatMessage(role="user", content=prompt)
    ]

    resp = model.generate(messages)
    text = resp.content.strip()

    import json
    parsed = {"is_spam": None, "spam_reason": None, "category": None}
    try:
        parsed = json.loads(text)
    except json.JSONDecodeError:
        # Minimal fallback heuristic if model ever fails JSON
        lower = text.lower()
        is_spam = ("not spam" not in lower) and ("spam" in lower)
        reason = None
        if is_spam and "reason" in lower:
            try:
                reason = lower.split("reason", 1)[1].split("\n", 1)[0].strip(": -")
            except Exception:
                pass
        cat = None
        for c in ["inquiry", "complaint", "thank-you", "request", "info"]:
            if c in lower:
                cat = c
                break
        parsed = {"is_spam": is_spam, "spam_reason": reason, "category": cat}


    is_spam = bool(parsed.get("is_spam"))
    email_category = parsed.get("category")
    spam_reason = parsed.get("spam_reason")
    new_msgs = state["messages"] + [
        ChatMessage(role="user",      content=prompt),
        ChatMessage(role="assistant", content=resp.content),
    ]

    return {
        "is_spam": is_spam,
        "spam_reason": spam_reason,
        "email_category": email_category,
        "messages": new_msgs
    }

In [5]:
def handle_spam(state: EmailState):
    """Agent discards spam email with a note"""
    print(f"Agent has marked the email as spam. Reason: {state['spam_reason']}")
    print("The email has been moved to the spam folder.")
    
    return {}

def draft_response(state: EmailState):
    """Agent drafts a preliminary response for legitimate emails"""
    e = state["email"]
    category = state.get("email_category") or "general"

    prompt = f"""
    Draft a brief, polite, professional reply to the email below.
    Assume category: {category}.
    Return only the draft reply text (no JSON).

    From: {e['sender']}
    Subject: {e['subject']}
    Body: {e['body']}
    """.strip()

    resp = model.generate([{"role": "user", "content": prompt}])

    new_msgs = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": resp.content},
    ]
    return {"email_draft": resp.content, "messages": new_msgs}

def notify_me(state: EmailState):
    """
    Notify me when an email comes in along with its classification
    """
    e = state["email"]
    print(f"\nSir, new mail from {e['sender']} ({state['email_category']})")
    print(state["email_draft"])
    return {}

def route_email(state: EmailState) -> str:
    return "spam" if state["is_spam"] else "legitimate" 

In [6]:
g = StateGraph(EmailState)

g.add_node("read_email", read_email)
g.add_node("classify_email", classify_email)
g.add_node("handle_spam", handle_spam)
g.add_node("draft_response", draft_response)
g.add_node("notify_mr_wayne", notify_me)

g.add_edge(START, "read_email")
g.add_edge("read_email", "classify_email")
g.add_conditional_edges("classify_email", route_email,{"spam": "handle_spam", "legitimate": "draft_response"})
g.add_edge("handle_spam", END)
g.add_edge("draft_response", "notify_mr_wayne")
g.add_edge("notify_mr_wayne", END)

mail_bot = g.compile()


In [10]:
legit = {
    "sender": "john.smith@example.com",
    "subject": "Question about your services",
    "body": "Dear Mr Wayne, I was referred by a colleague …"
}
spam = {
    "sender": "winner@lottery-intl.com",
    "subject": "YOU HAVE WON $5,000,000!!!",
    "body": "CONGRATULATIONS! Click here…"
}

In [11]:
print("👉 Legitimate mail")
mail_bot.invoke({
    "email": legit,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": [],
})

print("\n👉 Spam mail")
# IMPORTANT: pass the state again
mail_bot.invoke({
    "email": spam,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": [],
})


👉 Legitimate mail
Alfred is processing an email from john.smith@example.com with subject: Question about your services
Agent has marked the email as spam. Reason: ": null,
The email has been moved to the spam folder.

👉 Spam mail
Alfred is processing an email from winner@lottery-intl.com with subject: YOU HAVE WON $5,000,000!!!
Agent has marked the email as spam. Reason: ": "lottery/scam",
The email has been moved to the spam folder.


{'email': {'sender': 'winner@lottery-intl.com',
  'subject': 'YOU HAVE WON $5,000,000!!!',
  'body': 'CONGRATULATIONS! Click here…'},
 'is_spam': True,
 'spam_reason': '": "lottery/scam",',
 'email_category': None,
 'email_draft': None,
 'messages': [ChatMessage(content='You are an email triage agent. Read the email and return a JSON object with:\n    - "is_spam": true|false\n    - "spam_reason": string or null\n    - "category": one of ["inquiry","complaint","thank-you","request","info"] or null\n\n    Email:\n    From: winner@lottery-intl.com\n    Subject: YOU HAVE WON $5,000,000!!!\n    Body: CONGRATULATIONS! Click here…\n    Return JSON only.', additional_kwargs={}, response_metadata={}, role='user'),
  ChatMessage(content='```json\n{\n  "is_spam": true,\n  "spam_reason": "Lottery/Scam",\n  "category": null\n}\n```', additional_kwargs={}, response_metadata={}, role='assistant')]}